In [1]:
"""

190 page 기준 8분 정도
휴일 때 대략 100~200 page
평일 때 대략 400~500 page 

한 페이지에 15 기사

"""

'\n\n190 page 기준 8분 정도\n휴일 때 대략 100~200 page\n평일 때 대략 400~500 page \n\n한 페이지에 15 기사\n\n'

In [2]:
from datetime import date
import pandas as pd
from tqdm import notebook
import requests
from bs4 import BeautifulSoup
import re

start_date = date(2022, 1, 1)
end_date = date(2022, 1, 2)


In [3]:
def create_soup(url):

    res = requests.get(url)
    res.raise_for_status()
    soup = BeautifulSoup(res.content, 'html.parser', from_encoding='cp949')
    return soup

In [4]:
def get_articles(year,month,day):

    
    date_data = pd.DataFrame()
    
    year = str(year)
    month = str(month).zfill(2)
    day = str(day).zfill(2)
    date = year+month+day
    
    
    # 마지막 페이지 번호 추출
    url = f"https://news.daum.net/breakingnews/society?page=999&regDate={date}"
    soup = create_soup(url)
    last_page = soup.find("em",attrs ={"class": "num_page"})
    last_page_num = int(re.sub(r'[^0-9]', '', last_page.text))

    
    
    for i in notebook.tqdm(range(1,last_page_num+1)):
        url = f"https://news.daum.net/breakingnews/society?page={i}&regDate={date}"
        soup = create_soup(url)

        
    
        # 뉴스 리스트가 포함된 속성 추출
        news_list_part = soup.find('ul', attrs={'class':'list_news2 list_allnews'})
        
        # COLUME : press
        press_list = []
        press = news_list_part.find_all('span', attrs={'class':'info_news'})
        for _span in press:
            press_list.append(_span.text.split()[0])
        
        # 뉴스 링크 리스트 추출
        # COLUME : link
        link_list = []
        news_link_list = news_list_part.find_all('a',attrs={"class":"link_thumb"})
        for news_link in news_link_list:
            link_list.append(news_link["href"])
        
        
        
        # COLUME : articleid
        articleid = []
        for link in link_list:
            articleid.append(link.split("/")[-1])
        
        
        
        # 뉴스 링크 리스트 추출
        # COLUME : link
        news_link_list = news_list_part.find_all('a')

        # 링크마다 요청
        # COLUME : title, content ,journalist
        title_list = []
        journalist_list = []
        content_list = []

        for link in link_list:
            url = link
            article_soup = create_soup(url)
            article_soup = article_soup.find("article")
            
            ### title
            title = article_soup.find("h3", attrs={'class':"tit_view"})
            title_list.append(title.text)
            
            ### journalist
            journalist = article_soup.find("span", attrs={'class':"txt_info"})
            journalist_list.append(journalist.text)
            
            ### content
            content = article_soup.find("div", attrs={'class':"article_view"})
            
            if content.select_one("p.link_figure") != None:
                content.select_one("p.link_figure").decompose()
                
            
            news_contents = content.find_all("p")
            news_content= ""
            for _p in news_contents:
                news_content += _p.text
            news_content.replace("\xa0","")
            content_list.append(news_content)
            
                
                
        data =  pd.DataFrame(zip(articleid, title_list,journalist_list,press_list,content_list, link_list))
            
        date_data = pd.concat([date_data, data])
    
    return date_data

In [5]:
# from dateutil.rrule import rrule, DAILY

# df = pd.DataFrame()

# for date in notebook.tqdm(list(rrule(DAILY, dtstart=start_date, until=end_date))):
#     df = pd.concat([df, get_articles(date.year, date.month, date.day)])
# df.reset_index(drop=True, inplace=True)

# df = get_articles(date.year, date.month, date.day)

In [6]:
df = get_articles(2022, 1, 1)

  0%|          | 0/190 [00:00<?, ?it/s]

In [7]:
"""
HTTPSConnectionPool(host='v.daum.net', port=443): Max retries exceeded with url: /v/20220101121016727 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x0000020C9A0B23A0>: Failed to establish a new connection: [WinError 10060] 연결된 구성원으로부터 응답이 없어 연결하지 못했거나, 호스트로부터 응답이 없어 연결이 끊어졌습니다'))
"""


"\nHTTPSConnectionPool(host='v.daum.net', port=443): Max retries exceeded with url: /v/20220101121016727 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x0000020C9A0B23A0>: Failed to establish a new connection: [WinError 10060] 연결된 구성원으로부터 응답이 없어 연결하지 못했거나, 호스트로부터 응답이 없어 연결이 끊어졌습니다'))\n"

In [20]:
df.iloc[32][4]

"[앵커] 검찰과 공수처의 '대장동 개발 특혜'와 '고발 사주' 의혹 수사가 결국, 해를 넘겼습니다. 정치적 부담을 덜기 위해선 대선 전까지 결론을 내야 하는 만큼 수사기관의 부담이 더욱 커졌습니다. 김다연 기자가 보도합니다. [기자] 지난해 9월 검찰과 공수처는 각각 '대장동 개발 특혜'와 '고발 사주' 의혹 수사에 착수했습니다. 하지만 주요 관계자 사망과 잇단 신병확보 실패 등의 변수들로 차질이 생기면서 두 수사 모두 해를 넘겼습니다. 대장동 수사팀은 유동규, 김만배, 남욱, 정영학, 정민용 등 핵심 5인방을 재판에 넘긴 뒤 개발사업 보고 계통에 이름을 올렸던 성남시 관계자를 차례로 불러 조사하는 등 '윗선' 수사에 집중하는 듯했습니다. 하지만 초과이익 환수 조항 삭제에 관여한 것으로 알려진 유한기 전 개발사업본부장과 김문기 개발1처장이 숨지면서 수사에 제동이 걸렸습니다. 어수선한 분위기 속에 당시 성남시장이던 이재명 더불어민주당 대선 후보는 물론 최측근인 정진상 전 정책실장에 대해서도 조사 한 번 못해봤습니다. 화천대유 측으로부터 50억 원을 받거나 받기로 약속했다는 이른바 '50억 클럽' 등 로비 의혹 수사도 남은 과제입니다. 곽상도 전 의원에 대해선 구속영장이 한 차례 기각된 뒤 한 달째 신병처리 방향을 정하지 못하고 있고, 박영수 전 특검과 권순일 전 대법관 등 다른 인사와 관련해서도 아직 뚜렷한 혐의점이 발견되지 않은 것으로 알려졌습니다. 공수처의 '고발 사주' 수사도 매듭짓지 못한 채 공전하긴 마찬가지입니다. 고발장 작성과 전달에 관여한 혐의를 받는 손준성 검사와 김웅 국민의힘 의원 등 핵심인물이 불구속 기소되는 선에서 수사가 일단락될 거란 전망이 나왔지만, 손 검사 신병확보 시도가 잇따라 불발된 뒤 수사는 사실상 멈췄습니다. 당시 검찰총장으로 의혹의 최정점에 있는 윤석열 국민의힘 대선 후보에 대한 조사도 아직인데 김진욱 공수처장은 선거에 영향이 없도록 처리하겠다는 입장만 되풀이하고 있습니다. 유력 대선 후보 관련이라는 점에서 시작부터 관심이 집

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2742 entries, 0 to 13
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       2742 non-null   object
 1   1       2742 non-null   object
 2   2       2742 non-null   object
 3   3       2742 non-null   object
 4   4       2742 non-null   object
 5   5       2742 non-null   object
dtypes: object(6)
memory usage: 150.0+ KB
